In this post, I will go over tensorflow OpenCV object detection module. To be honest, I haven't used OpenCV for quite some time. And after recently looking into it, I have realized how awesome OpenCV has become. It <b>now has a dedicated DNN (deep neural network) module</b>. This module also has <b>functionality to load Caffe and Tensorflow trained networks.</b> I am just so happy to see that functionality in OpenCV. Just think about it, you can use your trained networks within OpenCV.

Alright, enough blabbering, let's get to the point. So in this post, I will use tensorflow detection module to load a trained tensorflow network and use this network to apply object detection to a webcam stream. So in the end, we will have a display that shows webcam stream and in the stream we modify the frames and display detected objects with rectangles. 

Alright, first thing is to get camera stream and display it. Later on we will expand this code to apply object detection to each frame instead of just displaying. 

In [1]:
#import sys

#print(sys.executable)
#print(sys.path)

#sys.path.append('/home/yutas/anaconda2/envs/testcaffe/lib/python2.7/site-packages/')

import cv2 as cv
#import tensorflow as tf

In [3]:
cam = cv.VideoCapture(0)

while True:
    ret_val, img = cam.read()

    cv.imshow('my webcam', img)

    if cv.waitKey(1) == 27: 
        break  # esc to quit

cv.destroyAllWindows()

error: OpenCV(3.4.1) /feedstock_root/build_artefacts/opencv_1520722613778/work/opencv-3.4.1/modules/highgui/src/window.cpp:356: error: (-215) size.width>0 && size.height>0 in function imshow


Now, we got the webcam stream working, next step is to integrate this with object detection. For object detection, I will use one of the networks available in Tensorflow Object detection module website : <a href='https://github.com/opencv/opencv/wiki/TensorFlow-Object-Detection-API'> Tensorflow Object Detection API</a>

In [4]:
cam = cv.VideoCapture(0)
cvNet = cv.dnn.readNetFromTensorflow('./mobilenet.pb', 'mobilenet.pbtxt')

while True:
    ret_val, img = cam.read()

    rows = img.shape[0]
    cols = img.shape[1]
    cvNet.setInput(cv.dnn.blobFromImage(img, 1.0/127.5, (300, 300), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    cvOut = cvNet.forward()

    for detection in cvOut[0,0,:,:]:
        score = float(detection[2])
        if score > 0.3:
            left = detection[3] * cols
            top = detection[4] * rows
            right = detection[5] * cols
            bottom = detection[6] * rows
            cv.rectangle(img, (int(left), int(top)), (int(right), int(bottom)), (23, 230, 210), thickness=2)

    cv.imshow('my webcam', img)

    if cv.waitKey(1) == 27: 
        break  # esc to quit

cam.release()
cv.destroyAllWindows()        
    